Install iapws

In [18]:
!pip install iapws

all imports

In [0]:
import numpy as np
from iapws import iapws97

SI units

In [0]:
class SI:
  
  SIpfx = {
      'G': 10**9,
      'M': 10**6,
      'k': 10**3,
      'h': 10**2,
      'da': 10**1,
      '1': 1,
      'd': 10**-1,
      'c': 10**-2,
      'm': 10**-3,
      'u': 10**-6,
    }
  
  def prefix(pfx):
    return SI.SIpfx.get(pfx)

Water properties

In [0]:
pa = 101325/10**6 #MPa
TK0 = 273.15

def WaterCp(T):
  return (iapws97._Region1(T+TK0, pa))["cp"] * 1000 #J/(kg*K)

def WaterDensity(T):
  return 1/iapws97._Region1(T+TK0, pa)["v"] #kg/m3

Definition of Class "Heat-transfer fluid"

In [0]:
class HTF:
  def __init__(self, temperature, volflow, type = 'water'):
    self.temperature = temperature
    self.volflow = volflow
    self.type = type
    
  def __str__(self):
    return '%s with %g °C, %g m3/s' % (self.type, self.temperature, self.volflow)
  
  def getCp(self):
    if self.type.lower() == 'water':
      return(WaterCp(self.temperature))
    else:
      return 0
    
  def getDens(self):
    if self.type.lower() == 'water':
      return(WaterDensity(self.temperature))
    else:
      return 0
    
  def mix(self, other):
    self.temperature = (self.temperature * self.volflow +
                        other.temperature * other.volflow) / (self.volflow + other.volflow)
    self.volflow = self.volflow + other.volflow

Definition of Class "Efficiency"

In [0]:
class Efficiency:
  def __init__(self, elecEff = 0, fuelEff = 0, htfEff = 0):
    self.elecEff = elecEff
    self.fuelEff = fuelEff
    self.htfEff = htfEff

Definition of power classes

In [0]:
class PowerType:
  def __init__(self, value = 0, SIprefix = '1'):
    self.value = value * SI.prefix(SIprefix)
    
  def __add__(self, other):
    return type(self)(self.value + other.value)
    
  def __str__(self):
    return '%g W' % (self.value)

class ElecPower(PowerType):
  pass

class FuelPower(PowerType):
  pass

class HtfPower(PowerType):
  pass

class Power:
  def __init__(self, *pwr_values):
    self.elecPower = ElecPower()
    self.fuelPower = FuelPower()
    self.htfPower = HtfPower()
    for pwr in pwr_values:
      if type(pwr) is ElecPower:
        self.elecPower += pwr
      elif type(pwr) is FuelPower:
        self.fuelPower += pwr
      elif type(pwr) is HtfPower:
        self.htfPower += pwr

  def all(self):
    return (self.elecPower.value + self.fuelPower.value + self.htfPower.value)
    
  def __mul__(self, other):
    if type(other) is Efficiency:
      return self.elecPower.value * other.elecEff + self.fuelPower.value * other.fuelEff + self.htfPower.value * other.htfEff
    else:
      return 0
    
  def __add__(self, other):
    return Power(self.elecPower + other.elecPower, self.fuelPower + other.fuelPower, self.htfPower + other.htfPower)
  
  def __str__(self):
    return '%g W' % (self.elecPower.value + self.fuelPower.value + self.htfPower.value)


Definition of heat transfer machines classes, definition forward task

ToDo: definition reverse task, Heat Pump

In [0]:
class HTMachine:
  # input values: Efficiency efficiency
  def __init__(self, efficiency):
    self.efficiency = efficiency
    self.genEff = 0
    self.htfout_min = 0
    self.htfout_max = 0
    self.pwrin_min = 0
    self.pwrin_max = 0

  # input values: Power powerin, HTF htfin
  # output values: Power powerout, HTF htfout
  def forwardtask(self, powerin, htfin):
    htfout = htfin
    htfout.temperature += powerin * self.efficiency / (htfin.volflow * htfin.getCp() * htfin.getDens())
    powerout = Power(ElecPower(powerin.fuelPower.value * self.genEff))
    return [powerout, htfout]

  #def reversetask (self, t2, htfin):
  #  htfout = htfin
  #  powerin = htfin.volflow * WaterCp(htfin.temperature)*1000 * WaterDensity(htfin.temperature)*(htfout.temperature-htfin.temperature)
  #  t2 = Tout()
  #  return (temp)
  
# Power to heat
class P2H_Machine(HTMachine):
  # input values: float elecEff
  def __init__(self, elecEff):
    self.efficiency = Efficiency(elecEff = elecEff)
    self.genEff = 0
    
# Furnace
class F_Machine(HTMachine):
  # input values: float fuelEff
  def __init__(self, fuelEff):
    self.efficiency = Efficiency(fuelEff = fuelEff)
    self.genEff = 0

# Heat pump
class HP_Machine(HTMachine):
  pass

# Cogeneration / combined heat and power
class CHP_Machine(HTMachine):
  # input values: float fuelEff, float genEff
  def __init__(self, fuelEff, genEff):
    self.efficiency = Efficiency(fuelEff = fuelEff)
    self.genEff = genEff
    
# Bypass
class BP_Machine(HTMachine):
  def __init__(self):
    self.efficiency = Efficiency()
    self.genEff = 0
    self.pwrin_max = Power()

Coupling variations

In [0]:
def computeParallel(htfin, machines):
  #check volume flow
  htfout_max = HTF(0,0)
  for machine in machines:
    #print("Adding " + str(machine.htfout_max) + " to " + str(htfout_max))
    htfout_max.mix(machine.htfout_max)
    #print("Result " + str(htfout_max))
    #print("")
    
  if htfout_max.volflow < htfin.volflow:
    raise ValueError("volume flow is too big")
    
  volflow_coeff = htfin.volflow / htfout_max.volflow
  
  htfout = HTF(0,0) 
  pwrout = Power()
  
  for machine in machines:
    pwrin_machine = Power(machine.pwrin_max)
    print("    pwrin_machine " + str(pwrin_machine))
    htfin_machine = HTF(htfin.temperature, machine.htfout_max.volflow * volflow_coeff)
    print("    htfin_machine " + str(htfin_machine))
    
    [pwrout_machine, htfout_machine] = machine.forwardtask(pwrin_machine, htfin_machine)
    
    print("    pwrout_machine " + str(pwrout_machine))
    print("    htfout_machine " + str(htfout_machine))
    
    pwrout = pwrout + pwrout_machine
    
    print("    Adding " + str(htfout_machine) + " to " + str(htfout))
    htfout.mix(htfout_machine)
    print("    Result " + str(htfout))
    print("")
    
  return [pwrout, htfout]

def computeSeries(htfin, stages):
  pwrout = Power()
  
  htfin_stage = htfin
  
  for stage in stages:
    print("  htfin_stage " + str(htfin_stage))
    [pwrout_stage, htfout_stage] = computeParallel(htfin_stage, stage)
    print("  pwrout_stage " + str(pwrout_stage))
    print("  htfout_stage " + str(htfout_stage))
    print("")
    
    pwrout += pwrout_stage
    htfin_stage = htfout_stage
    
  htfout = htfout_stage
  return [pwrout, htfout]

forward task

In [27]:
# Example configuration

htfin=HTF(70,0.077)

p2h_1 = P2H_Machine(1.0)
p2h_1.pwrin_max = ElecPower(100,"k")
p2h_1.htfout_max = HTF(999, 0.1)

p2h_2 = P2H_Machine(1.0)
p2h_2.pwrin_max = ElecPower(50,"k")
p2h_1.htfout_max = HTF(999, 0.1)

fur_1 = F_Machine(0.92)
fur_1.pwrin_max = FuelPower(6.52, "M")
fur_1.htfout_max = HTF(999, 0.057)

fur_2 = F_Machine(0.92)
fur_2.pwrin_max = FuelPower(5.44, "M")
fur_2.htfout_max = HTF(999, 0.047)

chp_1 = CHP_Machine(0.489, 0.411)
chp_1.pwrin_max = FuelPower(2, "M")
chp_1.htfout_max = HTF(999, 0.010)

chp_2 = CHP_Machine(0.499, 0.407)
chp_2.pwrin_max = FuelPower(2.42, "M")
chp_2.htfout_max = HTF(999, 0.0103)

bps_1 = BP_Machine()
bps_1.htfout_max = HTF(999, 0.01)

bps_2 = BP_Machine()
bps_2.htfout_max = HTF(999, 0.0197)

print("htfin " + str(htfin))

stage_1 = [chp_1, fur_1, bps_1]
stage_2 = [chp_2, fur_2, bps_2]

[pwrout, htfout] = computeSeries(htfin, [stage_1, stage_2])
print("pwrout " + str(pwrout))
print("htfout " + str(htfout))


htfin water with 70 °C, 0.077 m3/s
  htfin_stage water with 70 °C, 0.077 m3/s
    pwrin_machine 2e+06 W
    htfin_machine water with 70 °C, 0.01 m3/s
    pwrout_machine 822000 W
    htfout_machine water with 93.8826 °C, 0.01 m3/s
    Adding water with 93.8826 °C, 0.01 m3/s to water with 0 °C, 0 m3/s
    Result water with 93.8826 °C, 0.01 m3/s

    pwrin_machine 6.52e+06 W
    htfin_machine water with 70 °C, 0.057 m3/s
    pwrout_machine 0 W
    htfout_machine water with 95.6982 °C, 0.057 m3/s
    Adding water with 95.6982 °C, 0.057 m3/s to water with 93.8826 °C, 0.01 m3/s
    Result water with 95.4272 °C, 0.067 m3/s

    pwrin_machine 0 W
    htfin_machine water with 70 °C, 0.01 m3/s
    pwrout_machine 0 W
    htfout_machine water with 70 °C, 0.01 m3/s
    Adding water with 70 °C, 0.01 m3/s to water with 95.4272 °C, 0.067 m3/s
    Result water with 92.125 °C, 0.077 m3/s

  pwrout_stage 822000 W
  htfout_stage water with 92.125 °C, 0.077 m3/s

  htfin_stage water with 92.125 °C, 0.077 m